# High-level RNN Keras (CNTK) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "cntk"
import keras as K
import cntk
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, CuDNNGRU
from common.params_lstm import *
from common.utils import *

Using CNTK backend


In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print(K.backend.backend())
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.1.4
Numpy:  1.14.1
CNTK:  2.4
cntk
channels_last
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [4]:
def create_symbol(CUDNN=True, maxf=MAXFEATURES, edim=EMBEDSIZE, nhid=NUMHIDDEN, maxl=MAXLEN):
    model = Sequential()
    model.add(Embedding(maxf, edim, input_length=maxl))
    # Only return last output
    if not CUDNN:
        model.add(GRU(nhid, return_sequences=False, return_state=False))
    else:
        model.add(CuDNNGRU(nhid, return_sequences=False, return_state=False))
    model.add(Dense(2, activation='softmax'))
    return model

In [5]:
def init_model(m, lr=LR, b1=BETA_1, b2=BETA_2, eps=EPS):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.Adam(lr, b1, b2, eps),
        metrics = ['accuracy'])
    return m

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Trimming to 30000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000, 2) (25000, 2)
int32 int32 int32 int32
CPU times: user 5.39 s, sys: 448 ms, total: 5.84 s
Wall time: 5.84 s


In [7]:
%%time
# Load symbol
# CuDNN RNNs are only available with the TensorFlow backend.
sym = create_symbol(CUDNN=False)

CPU times: user 428 ms, sys: 554 ms, total: 982 ms
Wall time: 597 ms


In [8]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 22.7 ms, sys: 61.6 ms, total: 84.3 ms
Wall time: 7.05 ms


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 125)          3750000   
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               67800     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 3,818,002
Trainable params: 3,818,002
Non-trainable params: 0
_________________________________________________________________


In [10]:
%%time
# Main training loop: 53s
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/3
25000/25000 [==============================] - 18s 719us/step - loss: 0.5204 - acc: 0.7269
Epoch 2/3
25000/25000 [==============================] - 18s 709us/step - loss: 0.2381 - acc: 0.9092
Epoch 3/3
25000/25000 [==============================] - 18s 706us/step - loss: 0.1323 - acc: 0.9545
CPU times: user 53.2 s, sys: 1.14 s, total: 54.3 s
Wall time: 53.4 s


In [11]:
%%time
# Main evaluation loop: 7s
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 7.35 s, sys: 52.2 ms, total: 7.4 s
Wall time: 7.4 s


In [12]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.86076
